In [1]:
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class TreeNode:
    def __init__(self, attribute=None, threshold=None, value=None, condition=None, description=None):
        self.attribute = attribute
        self.threshold = threshold
        self.value = value
        self.condition = condition
        self.description = description
        self.children = {}

    def add_child(self, key, child_node):
        self.children[key] = child_node

    def is_leaf(self):
        return self.value is not None

    def predict(self, data, detail=None):
        if detail is None:
            detail = {}

        if self.is_leaf():
            alasan_parts = []

            # IPK
            ipk = data['IPK']
            if ipk < 3.0:
                alasan_parts.append("IPK rendah")
            elif 3.0 <= ipk <= 3.5:
                alasan_parts.append("IPK sedang")
            elif 3.51 <= ipk <= 3.75:
                alasan_parts.append("IPK cukup tinggi")
            else:
                alasan_parts.append("IPK tinggi")

            # TOEFL
            toefl = data['Toefl']
            if toefl < 450:
                alasan_parts.append("TOEFL rendah")
            elif 450 <= toefl < 500:
                alasan_parts.append("TOEFL cukup")
            elif 500 <= toefl < 600:
                alasan_parts.append("TOEFL tinggi")
            else:
                alasan_parts.append("TOEFL sangat tinggi")

            # Prestasi
            prestasi = data['Skor_Prestasi']
            if prestasi < 4:
                alasan_parts.append("Prestasi rendah")
            elif 4 <= prestasi < 6:
                alasan_parts.append("Prestasi sedang")
            else:
                alasan_parts.append("Prestasi tinggi")

            # Organisasi
            organisasi = data['Skor_Organisasi']
            if organisasi < 4:
                alasan_parts.append("Organisasi tidak aktif")
            elif 4 <= organisasi < 8:
                alasan_parts.append("Organisasi kurang aktif")
            else:
                alasan_parts.append("Organisasi aktif")

            return self.value, ', '.join(alasan_parts)

        if self.attribute == 'IPK':
            ipk = data['IPK']
            if ipk < 3.0:
                return self.children['<3'].predict(data, detail)
            elif 3.0 <= ipk <= 3.5:
                return self.children['3-3.5'].predict(data, detail)
            elif 3.51 <= ipk <= 3.75:
                return self.children['3.51-3.75'].predict(data, detail)
            else:
                return self.children['3.76-4.00'].predict(data, detail)

        elif self.attribute == 'Toefl':
            condition = 'high' if data['Toefl'] >= self.threshold else 'low'
            return self.children[condition].predict(data, detail)

        elif self.attribute == 'Prestasi':
            condition = 'high' if data['Skor_Prestasi'] >= self.threshold else 'low'
            return self.children[condition].predict(data, detail)

        elif self.attribute == 'Organisasi':
            condition = 'high' if data['Skor_Organisasi'] >= self.threshold else 'low'
            return self.children[condition].predict(data, detail)

        return "Tidak Lolos", "Atribut tidak dikenali"


def skor_ipk(ipk):
    if 3.0 <= ipk <= 3.5: return 5
    elif 3.51 <= ipk <= 3.75: return 7
    elif 3.76 <= ipk <= 4.0: return 10
    return 0

def skor_toefl(toefl):
    if 400 <= toefl <= 492: return 3
    elif 493 <= toefl <= 585: return 5
    elif 586 <= toefl <= 677: return 7
    return 0

def skor_prestasi(prestasi):
    prestasi = str(prestasi).lower()
    skor = 0
    if 'internasional' in prestasi: skor += 6
    if 'nasional' in prestasi: skor += 5
    if 'provinsi' in prestasi: skor += 3
    if 'kabupaten' in prestasi or 'kota' in prestasi: skor += 2
    if 'juara 1' in prestasi: skor += 6
    if 'juara 2' in prestasi: skor += 5
    if 'juara 3' in prestasi: skor += 4
    if 'harapan 1' in prestasi: skor += 3
    if 'harapan 2' in prestasi: skor += 2
    if 'harapan 3' in prestasi: skor += 1
    return skor

def skor_organisasi(internal, eksternal, peran):
    skor = 0
    if isinstance(internal, str): skor += len([x for x in internal.split(',') if x.strip()]) * 3
    if isinstance(eksternal, str): skor += len([x for x in eksternal.split(',') if x.strip()]) * 5
    if isinstance(peran, str):
        p = peran.lower()
        if "ketua" in p or "presiden" in p: skor += 5
        elif "wakil" in p or "sekretaris" in p or "bendahara" in p: skor += 3
        elif "anggota" in p or "staf" in p: skor += 1
    return skor

def build_tree():
    root = TreeNode(attribute='IPK')
    root.add_child('<3', TreeNode(value='Tidak Lolos', description='IPK di bawah 3.0'))

    for ipk_range, threshold in [('3-3.5', 475), ('3.51-3.75', 450), ('3.76-4.00', 425)]:
        ipk_node = TreeNode(attribute='Toefl', threshold=threshold)
        root.add_child(ipk_range, ipk_node)

        toefl_high = TreeNode(attribute='Prestasi', threshold=6)
        toefl_low = TreeNode(value='Tidak Lolos', description='TOEFL rendah')
        ipk_node.add_child('high', toefl_high)
        ipk_node.add_child('low', toefl_low)

        prestasi_high = TreeNode(attribute='Organisasi', threshold=6)
        prestasi_low = TreeNode(value='Tidak Lolos', description='Prestasi rendah')
        toefl_high.add_child('high', prestasi_high)
        toefl_high.add_child('low', prestasi_low)

        prestasi_high.add_child('high', TreeNode(value='Lolos', description='Memenuhi semua syarat'))
        prestasi_high.add_child('low', TreeNode(value='Tidak Lolos', description='Organisasi rendah'))

    return root

def visualize_tree_matplotlib(root):
    fig, ax = plt.subplots(figsize=(15, 10))
    ax.set_aspect('equal')
    ax.axis('off')
    node_positions = {}
    level_nodes = {}
    def get_tree_height(node):
        if not node.children:
            return 1
        return 1 + max(get_tree_height(child) for child in node.children.values())
    max_height = get_tree_height(root)
    def assign_positions(node, level, x_offset):
        if level not in level_nodes:
            level_nodes[level] = []
        level_nodes[level].append(node)
        if not node.children:
            node_positions[node] = (x_offset, -level)
            return x_offset + 1
        current_x = x_offset
        child_x_coords = []
        for child_condition, child_node in node.children.items():
            current_x = assign_positions(child_node, level + 1, current_x)
            child_x_coords.append(node_positions[child_node][0])
        node_x = sum(child_x_coords) / len(child_x_coords) if child_x_coords else x_offset
        node_positions[node] = (node_x, -level)
        return current_x
    assign_positions(root, 0, 0)
    min_x = min(pos[0] for pos in node_positions.values())
    max_x = max(pos[0] for pos in node_positions.values())
    center_offset = (max_x - min_x) / 2
    for node, pos in node_positions.items():
        node_positions[node] = (pos[0] - center_offset, pos[1])
    node_width = 0.8
    node_height = 0.4
    for node, (x, y) in node_positions.items():
        if node.value == 'Lolos': color = 'lightgreen'
        elif node.value == 'Tidak Lolos': color = 'salmon'
        else: color = 'lightblue'
        rect = patches.Rectangle((x - node_width/2, y - node_height/2), node_width, node_height,
                                 facecolor=color, edgecolor='black', linewidth=1.5)
        ax.add_patch(rect)
        label = ''
        if node.attribute:
            label += f'{node.attribute}'
            if node.threshold is not None:
                label += f'\n(>{node.threshold})'
        elif node.value:
            label += f'{node.value}'
            if node.description:
                label += f'\n({node.description})'
        ax.text(x, y, label, ha='center', va='center', fontsize=10, wrap=True)
        for condition, child_node in node.children.items():
            child_x, child_y = node_positions[child_node]
            ax.plot([x, child_x], [y - node_height/2, child_y + node_height/2], 'k-')
            mid_x = (x + child_x) / 2
            mid_y = (y - node_height/2 + child_y + node_height/2) / 2
            ax.text(mid_x, mid_y, condition, ha='center', va='center', fontsize=9,
                    bbox=dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.2'))
    plt.tight_layout()
    plt.savefig('decision_tree_combined.png')
    plt.close(fig)

def proses_data(file_path):
    df = pd.read_excel(file_path)
    hasil = []

    for _, row in df.iterrows():
        peserta = {
            'IPK': row['IPK'],
            'Toefl': row['Toefl'],
            'Skor_Prestasi': skor_prestasi(row['Prestasi']),
            'Skor_Organisasi': skor_organisasi(row['Organisasi Internal'], row['Organisasi Eksternal'], row['Organisasi & Peran']),
        }

        keputusan, alasan = build_tree().predict(peserta)
        skor_total = (
            skor_ipk(peserta['IPK']) +
            skor_toefl(peserta['Toefl']) +
            peserta['Skor_Prestasi'] +
            peserta['Skor_Organisasi']
        )

        hasil.append({
            'Nama': row['Nama'],
            'IPK': peserta['IPK'],
            'Toefl': peserta['Toefl'],
            'Skor_IPK': skor_ipk(peserta['IPK']),
            'Skor_TOEFL': skor_toefl(peserta['Toefl']),
            'Skor_Prestasi': peserta['Skor_Prestasi'],
            'Skor_Organisasi': peserta['Skor_Organisasi'],
            'Total Skor': skor_total,
            'Status': keputusan,
            'Alasan': alasan
        })

    return pd.DataFrame(hasil)

def menu_interaktif(df_hasil):
    while True:
        print("\n===== MENU INTERAKTIF =====")
        print("1. Tampilkan Peserta Lolos Seleksi")
        print("2. Cari Peserta berdasarkan Nama")
        print("3. Tampilkan Semua Peserta")
        print("4. Visualisasi Decision Tree")
        print("5. Keluar")
        pilih = input("\nPilihan (1/2/3/4/5): ")

        if pilih == '1':
            lolos_df = df_hasil[df_hasil['Status'] == 'Lolos'].sort_values(by='Total Skor', ascending=False).head(20)
            print("\n        ----- Nama Nama Peserta Lolos -----")
            print(lolos_df[['Nama', 'Total Skor', 'Status']].to_string(index=False))
            print("\n >> Hasil disimpan di 'Hasil_Semua_Seleksi.xlsx'")

        elif pilih == '2':
            nama = input("\nMasukkan nama peserta: ").lower()
            hasil = df_hasil[df_hasil['Nama'].str.lower().str.contains(nama)]
            if not hasil.empty:
                print(hasil[['Nama','Total Skor', 'Status']].to_string(index=False))
            else:
                print("Peserta tidak ditemukan.")

        elif pilih == '3':
            print(df_hasil.sort_values(by='Total Skor', ascending=False)[['Nama', 'Total Skor', 'Status']].to_string(index=False))

        elif pilih == '4':
            print("\nMembuat visualisasi decision tree...")
            tree = build_tree()
            visualize_tree_matplotlib(tree)
            print("Visualisasi selesai. File telah disimpan sebagai 'decision_tree_combined.png'.")

        elif pilih == '5':
            print("Keluar.")
            break
        else:
            print("Pilihan tidak valid.")

def main():
    import sys
    import os

    is_colab = 'google.colab' in sys.modules

    if is_colab:
        from google.colab import files
        print("Upload file Excel data pendaftar beasiswa:")
        uploaded = files.upload()
        if not uploaded:
            print("Tidak ada file yang diupload.")
            return
        file_path = list(uploaded.keys())[0]
    else:
        default_path = "Data_Dummy_Kel2_2024A.xlsx"
        file_path = input(f"Masukkan path file Excel data pendaftar beasiswa (default: {default_path}): ") or default_path
        if not os.path.exists(file_path):
            print(f"File '{file_path}' tidak ditemukan.")
            return

    df_hasil = proses_data(file_path)
    df_hasil.to_excel("Hasil_Semua_Seleksi.xlsx", index=False)
    print(">> Semua hasil seleksi disimpan di 'Hasil_Seleksi_Semua.xlsx'")
    menu_interaktif(df_hasil)

if __name__ == "__main__":
    main()



Upload file Excel data pendaftar beasiswa:


Saving Data_Dummy_Kel2_2024A.xlsx to Data_Dummy_Kel2_2024A.xlsx
>> Semua hasil seleksi disimpan di 'Hasil_Seleksi_Semua.xlsx'

===== MENU INTERAKTIF =====
1. Tampilkan Peserta Lolos Seleksi
2. Cari Peserta berdasarkan Nama
3. Tampilkan Semua Peserta
4. Visualisasi Decision Tree
5. Keluar

Pilihan (1/2/3/4/5): 5
Keluar.
